In [ ]:
# | default_exp blocks/se

# Imports

In [ ]:
# | export


import torch
from torch import nn

from vision_architectures.blocks.cnn import CNNBlock3D
from vision_architectures.utils.activation_checkpointing import ActivationCheckpointing
from vision_architectures.utils.custom_base_model import CustomBaseModel
from vision_architectures.utils.rearrange import rearrange_channels

# Config

In [ ]:
# | export


class SEBlock3DConfig(CustomBaseModel):
    dim: int
    r: float
    normalization: str = "batchnorm3d"
    activation: str = "silu"

# Architecture

In [ ]:
# | export


class SEBlock3D(nn.Module):
    def __init__(self, config: SEBlock3DConfig = {}, checkpointing_level: int = 0, **kwargs):
        super().__init__()

        self.config = SEBlock3DConfig.model_validate(config | kwargs)

        dim = self.config.dim
        r = self.config.r

        excitation_dim = int(dim // r)

        self.squeeze = nn.AdaptiveAvgPool3d((1, 1, 1))
        self.excite = nn.Sequential(
            CNNBlock3D(
                self.config.model_dump()
                | {
                    "in_channels": dim,
                    "out_channels": excitation_dim,
                    "kernel_size": 1,
                },
                checkpointing_level,
            ),
            CNNBlock3D(
                self.config.model_dump()
                | {
                    "in_channels": excitation_dim,
                    "out_channels": dim,
                    "kernel_size": 1,
                    "activation": "sigmoid",
                },
                checkpointing_level,
            ),
        )

        self.checkpointing_level2 = ActivationCheckpointing(2, checkpointing_level)

    def _forward(self, x: torch.Tensor, channels_first: bool = True):
        # x: (b, [dim], z, y, x, [dim])

        x = rearrange_channels(x, channels_first, True)
        # Now x is (b, dim, z, y, x)

        p = self.squeeze(x)
        # (b, dim, 1, 1, 1)
        p = self.excite(p)
        # (b, dim, 1, 1, 1)
        x = x * p
        # (b, dim, z, y, x)

        x = rearrange_channels(x, True, channels_first)
        # (b, [dim], z, y, x, [dim])

        return x

    def forward(self, *args, **kwargs):
        return self.checkpointing_level2(self._forward, *args, **kwargs)

In [ ]:
test = SEBlock3D(dim=12, r=3)
display(test)

sample_input = torch.randn(2, 12, 4, 4, 4)
test(sample_input).shape


SEBlock3D(
  (squeeze): AdaptiveAvgPool3d(output_size=(1, 1, 1))
  (excite): Sequential(
    (0): CNNBlock3D(
      (cnn): Conv3d(12, 4, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=same, bias=False)
      (norm_layer): BatchNorm3d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_layer): SiLU()
      (dropout): Dropout(p=0.0, inplace=False)
      (checkpointing_level1): ActivationCheckpointing(enabled=False, checkpointing_level=1)
    )
    (1): CNNBlock3D(
      (cnn): Conv3d(4, 12, kernel_size=(1, 1, 1), stride=(1, 1, 1), padding=same, bias=False)
      (norm_layer): BatchNorm3d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_layer): Sigmoid()
      (dropout): Dropout(p=0.0, inplace=False)
      (checkpointing_level1): ActivationCheckpointing(enabled=False, checkpointing_level=1)
    )
  )
  (checkpointing_level2): ActivationCheckpointing(enabled=False, checkpointing_level=2)
)

torch.Size([2, 12, 4, 4, 4])

# nbdev

In [ ]:
!nbdev_export